In [60]:
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import text
from connection import connect

In [2]:
co_oltp, etl_conn, etl_conn_or = connect()

# Extract

In [7]:
# Tomando valores de los años
query = text("""
    SELECT
        c.customer_id,
        EXTRACT(YEAR FROM MIN(soh.order_date)) AS first_order_year,
        EXTRACT(YEAR FROM MAX(soh.order_date)) AS last_order_year,
        EXTRACT(MONTH FROM MAX(soh.order_date)) AS order_month,
        -- OrderFrequency es complejo, lo podemos dejar como un valor fijo por ahora.
        'Variable' as order_frequency
    FROM sales.customer AS c
    INNER JOIN sales.sales_order_header AS soh ON c.customer_id = soh.customer_id
    -- Asegurarnos de que solo tomamos clientes que son revendedores
    WHERE c.store_id IS NOT NULL
    GROUP BY c.customer_id;
""")
t_years = pd.read_sql(query, co_oltp)

In [8]:
t_years

,customer_id,first_order_year,last_order_year,order_month,order_frequency
0,29873,2012.0,2014.0,5.0,Variable
1,30052,2011.0,2012.0,2.0,Variable
2,29712,2012.0,2014.0,3.0,Variable
3,29744,2012.0,2014.0,3.0,Variable
4,29733,2011.0,2012.0,1.0,Variable
...,...,...,...,...,...
630,29501,2013.0,2014.0,5.0,Variable
631,29587,2012.0,2014.0,3.0,Variable
632,29654,2012.0,2014.0,3.0,Variable
633,30022,2011.0,2014.0,3.0,Variable


In [111]:
query_reseller = text("""
-- Usamos una CTE para pre-agregar la información de los clientes por tienda
-- CTE para pre-agregar información de clientes y órdenes (como antes)
WITH ResellerCustomerInfo AS (
    SELECT
        c.store_id,
        MIN(c.account_number) AS reseller_alternate_key,
        EXTRACT(YEAR FROM MIN(soh.order_date)) AS first_order_year,
        EXTRACT(YEAR FROM MAX(soh.order_date)) AS last_order_year,
        EXTRACT(MONTH FROM MAX(soh.order_date)) AS order_month,
        '0' as order_frequency
    FROM sales.customer AS c
    LEFT JOIN sales.sales_order_header AS soh ON c.customer_id = soh.customer_id
    WHERE c.store_id IS NOT NULL
    GROUP BY c.store_id
),
-- CTE para seleccionar UNA ÚNICA dirección por tienda (Revendedor)
RankedAddresses AS (
    SELECT
        bea.business_entity_id,
        a.address_line_1,
        a.address_line_2,
        a.city,
        a.postal_code,
        st.name AS state_province_name,
        cr.name AS country_region_name,
        -- Rankeamos las direcciones por tipo, priorizando 'Main Office'
        ROW_NUMBER() OVER(
            PARTITION BY bea.business_entity_id
            ORDER BY
                CASE
                    WHEN at.name = 'Main Office' THEN 1
                    ELSE 99
                END
        ) as rn_addr
    FROM person.business_entity_address AS bea
    JOIN person.address AS a ON bea.address_id = a.address_id
    JOIN person.address_type AS at ON bea.address_type_id = at.address_type_id
    JOIN person.state_province AS st ON a.state_province_id = st.state_province_id
    JOIN person.country_region AS cr ON st.country_region_code = cr.country_region_code
),
-- CTE para seleccionar UN ÚNICO teléfono por tienda, a través del contacto
RankedContacts AS (
    SELECT
        bec.business_entity_id, -- El ID de la tienda (Store)
        pp.phone_number,
        -- Rankeamos los contactos, por si hubiera más de uno
        ROW_NUMBER() OVER(
            PARTITION BY bec.business_entity_id
            ORDER BY bec.contact_type_id -- Podríamos priorizar un tipo de contacto si fuera necesario
        ) as rn_contact
    FROM person.business_entity_contact AS bec
    JOIN person.person AS p ON bec.person_id = p.business_entity_id
    JOIN person.person_phone AS pp ON p.business_entity_id = pp.business_entity_id
)
SELECT
    s.business_entity_id,
    s.name AS reseller_name,
    s.demographics,

    -- Datos de la dirección desde la CTE de direcciones
    ra.address_line_1 AS address_line1,
    ra.address_line_2 AS address_line2,
    ra.city,
    ra.postal_code,
    ra.state_province_name,
    ra.country_region_name,

    -- Datos del cliente pre-agregados desde la primera CTE
    rci.reseller_alternate_key,
    rci.first_order_year,
    rci.last_order_year,
    rci.order_month,
    rci.order_frequency,


    -- El teléfono desde la CTE de contactos
    rc.phone_number AS phone

FROM sales.store AS s

-- LEFT JOIN a la CTE de info de clientes
LEFT JOIN ResellerCustomerInfo AS rci ON s.business_entity_id = rci.store_id

-- LEFT JOIN a la CTE de direcciones, filtrando solo por la principal (rn_addr = 1)
LEFT JOIN RankedAddresses AS ra
    ON s.business_entity_id = ra.business_entity_id
    AND ra.rn_addr = 1

-- LEFT JOIN a la CTE de contactos, filtrando solo por el principal (rn_contact = 1)
LEFT JOIN RankedContacts AS rc
    ON s.business_entity_id = rc.business_entity_id
    AND rc.rn_contact = 1;
""")

df_reseller = pd.read_sql(query_reseller, co_oltp)
print(f"Registros extraidos: {len(df_reseller)}")
df_reseller

Registros extraidos: 701


,business_entity_id,reseller_name,demographics,address_line1,address_line2,city,postal_code,state_province_name,country_region_name,reseller_alternate_key,first_order_year,last_order_year,order_month,order_frequency,phone
0,292,Next-Door Bike Store,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Mall Of Memphis,None,Memphis,38103,Tennessee,United States,AW00000585,2011.0,2013.0,4.0,0,398-555-0132
1,294,Professional Sales and Service,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",57251 Serene Blvd,None,Van Nuys,91411,California,United States,AW00000582,2013.0,2014.0,5.0,0,747-555-0171
2,296,Riders Company,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Tanger Factory,None,Branch,55056,Minnesota,United States,AW00000579,2011.0,2014.0,5.0,0,334-555-0137
3,298,The Bike Mechanics,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Johnny Appleseed Shop.center,None,Mansfield,44903,Ohio,United States,AW00000576,2011.0,2014.0,5.0,0,599-555-0127
4,300,Nationwide Supply,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",4250 Concord Road,None,Rhodes,2138,New South Wales,Australia,AW00000573,2013.0,2014.0,5.0,0,1 (11) 500 555-0132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,1988,Retreat Inn,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Suite 2502 410 Albert Street,None,Waterloo,N2V,Ontario,Canada,AW00000334,2012.0,2014.0,3.0,0,155-555-0140
697,1990,Technical Parts Manufacturing,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",Ontario Mills,None,Ontario,91764,California,United States,AW00000331,2013.0,2014.0,3.0,0,433-555-0168
698,1992,Totes & Baskets Company,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",72540 Blanco Rd.,None,San Antonio,78204,Texas,United States,AW00000328,2011.0,2014.0,3.0,0,560-555-0171
699,1994,World of Bikes,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",660 Lindbergh,None,Saint Louis,63103,Missouri,United States,AW00000327,2012.0,2014.0,5.0,0,695-555-0158


In [112]:
df_reseller.columns

Index(['business_entity_id', 'reseller_name', 'demographics', 'address_line1',
       'address_line2', 'city', 'postal_code', 'state_province_name',
       'country_region_name', 'reseller_alternate_key', 'first_order_year',
       'last_order_year', 'order_month', 'order_frequency', 'phone'],
      dtype='object')

# Transform

In [113]:
def parse_store_demographics(xml_string):
    if pd.isna(xml_string):
        return {}

    try:
        root = ET.fromstring(xml_string)
        ns = {'ns': root.tag.split('}')[0].strip('{')}

        def namespace_find(tag):
            el = root.find(f'ns:{tag}', ns)
            return el.text if el is not None else None


        result = {
            'annual_sales': namespace_find('AnnualSales'),
            'annual_revenue': namespace_find('AnnualRevenue'),
            'bank_name': namespace_find('BankName'),
            'business_type': namespace_find('BusinessType'),
            'year_opened': namespace_find('YearOpened'),
            'product_line': namespace_find('Specialty'),
            'number_employees': namespace_find('NumberEmployees'),
            'min_payment_type': None,
            'min_payment_amount': None,
        }
        return result

    except Exception as e:
        return {}


In [114]:
demographics_df = df_reseller['demographics'].apply(parse_store_demographics).apply(pd.Series)
df_with_demographics = pd.concat([df_reseller.drop('demographics', axis=1), demographics_df], axis=1)

In [115]:
df_with_demographics

,business_entity_id,reseller_name,address_line1,address_line2,city,postal_code,state_province_name,country_region_name,reseller_alternate_key,first_order_year,...,phone,annual_sales,annual_revenue,bank_name,business_type,year_opened,product_line,number_employees,min_payment_type,min_payment_amount
0,292,Next-Door Bike Store,Mall Of Memphis,None,Memphis,38103,Tennessee,United States,AW00000585,2011.0,...,398-555-0132,800000,80000,United Security,BM,1996,Mountain,13,None,None
1,294,Professional Sales and Service,57251 Serene Blvd,None,Van Nuys,91411,California,United States,AW00000582,2013.0,...,747-555-0171,800000,80000,International Bank,BM,1991,Touring,14,None,None
2,296,Riders Company,Tanger Factory,None,Branch,55056,Minnesota,United States,AW00000579,2011.0,...,334-555-0137,800000,80000,Primary Bank & Reserve,BM,1999,Road,15,None,None
3,298,The Bike Mechanics,Johnny Appleseed Shop.center,None,Mansfield,44903,Ohio,United States,AW00000576,2011.0,...,599-555-0127,800000,80000,International Security,BM,1994,Mountain,16,None,None
4,300,Nationwide Supply,4250 Concord Road,None,Rhodes,2138,New South Wales,Australia,AW00000573,2013.0,...,1 (11) 500 555-0132,800000,80000,Guardian Bank,BM,1987,Touring,17,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,1988,Retreat Inn,Suite 2502 410 Albert Street,None,Waterloo,N2V,Ontario,Canada,AW00000334,2012.0,...,155-555-0140,300000,30000,Primary Bank & Reserve,BM,1982,Road,8,None,None
697,1990,Technical Parts Manufacturing,Ontario Mills,None,Ontario,91764,California,United States,AW00000331,2013.0,...,433-555-0168,300000,30000,International Security,BM,1976,Touring,5,None,None
698,1992,Totes & Baskets Company,72540 Blanco Rd.,None,San Antonio,78204,Texas,United States,AW00000328,2011.0,...,560-555-0171,300000,30000,Guardian Bank,BM,1970,Road,2,None,None
699,1994,World of Bikes,660 Lindbergh,None,Saint Louis,63103,Missouri,United States,AW00000327,2012.0,...,695-555-0158,800000,80000,Primary Bank & Reserve,BM,1997,Mountain,17,None,None


In [116]:
# Vincula con DimGeography
df_geo_with_keys = pd.read_sql(
    text("""
        SELECT geography_key, city, postal_code, state_province_name, english_country_region_name as country_region_name
        FROM dim_geography;
    """),
    etl_conn
)

df_reseller_linked = pd.merge(
    df_with_demographics,
    df_geo_with_keys,
    on=['city', 'postal_code', 'state_province_name', 'country_region_name'], # <-- Añadimos postal_code
    how='left'
)

In [117]:
print("Despues del merge con DimGeography:", df_reseller_linked.shape)
df_reseller_linked

Despues del merge con DimGeography: (701, 24)


,business_entity_id,reseller_name,address_line1,address_line2,city,postal_code,state_province_name,country_region_name,reseller_alternate_key,first_order_year,...,annual_sales,annual_revenue,bank_name,business_type,year_opened,product_line,number_employees,min_payment_type,min_payment_amount,geography_key
0,292,Next-Door Bike Store,Mall Of Memphis,None,Memphis,38103,Tennessee,United States,AW00000585,2011.0,...,800000,80000,United Security,BM,1996,Mountain,13,None,None,40
1,294,Professional Sales and Service,57251 Serene Blvd,None,Van Nuys,91411,California,United States,AW00000582,2013.0,...,800000,80000,International Bank,BM,1991,Touring,14,None,None,545
2,296,Riders Company,Tanger Factory,None,Branch,55056,Minnesota,United States,AW00000579,2011.0,...,800000,80000,Primary Bank & Reserve,BM,1999,Road,15,None,None,156
3,298,The Bike Mechanics,Johnny Appleseed Shop.center,None,Mansfield,44903,Ohio,United States,AW00000576,2011.0,...,800000,80000,International Security,BM,1994,Mountain,16,None,None,577
4,300,Nationwide Supply,4250 Concord Road,None,Rhodes,2138,New South Wales,Australia,AW00000573,2013.0,...,800000,80000,Guardian Bank,BM,1987,Touring,17,None,None,592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,1988,Retreat Inn,Suite 2502 410 Albert Street,None,Waterloo,N2V,Ontario,Canada,AW00000334,2012.0,...,300000,30000,Primary Bank & Reserve,BM,1982,Road,8,None,None,465
697,1990,Technical Parts Manufacturing,Ontario Mills,None,Ontario,91764,California,United States,AW00000331,2013.0,...,300000,30000,International Security,BM,1976,Touring,5,None,None,319
698,1992,Totes & Baskets Company,72540 Blanco Rd.,None,San Antonio,78204,Texas,United States,AW00000328,2011.0,...,300000,30000,Guardian Bank,BM,1970,Road,2,None,None,271
699,1994,World of Bikes,660 Lindbergh,None,Saint Louis,63103,Missouri,United States,AW00000327,2012.0,...,800000,80000,Primary Bank & Reserve,BM,1997,Mountain,17,None,None,203


In [118]:
# Limpia valores
df_reseller_linked['business_type'] = df_reseller_linked['business_type'].fillna('Unknown')
df_reseller_linked['product_line'] = df_reseller_linked['product_line'].fillna('None')

In [119]:
# Columnas finales segun DimReseller del DW
final_columns = [
    'geography_key', 'reseller_alternate_key', 'phone', 'business_type',
    'reseller_name', 'number_employees', 'order_frequency', 'order_month',
    'first_order_year', 'last_order_year', 'product_line', 'address_line1',
    'address_line2', 'annual_sales', 'bank_name', 'min_payment_type',
    'min_payment_amount', 'annual_revenue', 'year_opened'
]

df_to_load = df_reseller_linked[final_columns]

# Load

In [120]:
df_to_load.to_sql(
    'dim_reseller',
    etl_conn,
    schema='dw',
    if_exists='append',
    index=False
)

701